# Setup Geral

Se estiver executando este exercício no Google Colab, execute as próximas duas células. 

Caso esteja executando localmente, não é necessário executar mas certifique-se de que o **pyspark** está instalado e configurado em sua máquina.

In [31]:
%%bash

# Instal Java
# apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
# pip install -q pyspark

In [1]:
%%bash 
# Create dir for data_cleansed
rm -rf bases_limpas
mkdir bases_limpas

In [33]:
# import os
# os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

# from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("local[*]").getOrCreate()

# Teste

O teste a ser realizado é composto de 3 partes:

- um exercício de programação em Python
- alguns exercícios de programação em SQL
- alguns exercícios de programação com PySpark

Você pode escolher qual das partes do exercício vai fazer primeiro. Todo o exercício deve ser completado no período de 90 minutos.

# Python

In [70]:
# SETUP
nomes_alunos = [
    ('Maria', 1),
    ('João', 2),
    ('Pedro', 3),
    ('Gabriella', 4),
    ('Giovana', 5),
    ('Arthur', 6)
]

notas_alunos = {
    1: 9.5,
    2: 5.1,
    3: 8.7,
    4: 7.1,
    5: 4.8,
    6: 6.3
}

Implemente uma função que recebe uma lista de nomes de alunos, um dicionário de notas dos mesmo, sendo que essas estruturas se relacionam por um ID.

A função deve retornar em ordem alfabética, o nome dos alunos que obtiveram notas maior ou igual de uma nota de corte informada.

In [87]:
def filtra_alunos_acima_corte(alunos, notas, nota_corte):

    alunos = sorted(dict(alunos).items(), key=lambda kv: (kv[0], kv[1]))
    alunos = dict(alunos)

    for aluno in alunos:
        if notas_alunos.get(alunos[aluno]) >= nota_corte:
            print(aluno)
    
filtra_alunos_acima_corte(nomes_alunos, notas_alunos, 6)

Arthur
Gabriella
Maria
Pedro


# SQL

**Setup**


In [2]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

mkdir: cannot create directory ‘bases_teste’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1039  100  1039    0     0   8056      0 --:--:-- --:--:-- --:--:--  8117
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533k  100  533k    0     0  2282k      0 --:--:-- --:--:-- --:--:-- 2291k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211k  100  211k    0     0  1547k      0 --:--:-- --:--:-- --:--:-- 1553k


In [3]:
# Encontrando o Spark Local
import findspark
findspark.init('/usr/local/spark/spark-3.3.0-bin-hadoop3/')
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

22/07/31 02:58:23 WARN Utils: Your hostname, LNVTP01X resolves to a loopback address: 127.0.1.1; using 172.29.227.219 instead (on interface eth0)
22/07/31 02:58:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/31 02:58:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/31 02:58:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/07/31 02:58:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [38]:
# Setup Spark Session
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

In [19]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

type(usuarios)

pyspark.sql.dataframe.DataFrame

**Função para execução de queries**

In [20]:
def q(query, n=30):
    return spark.sql(query).show(n=n, truncate=False)

Para executar alguma consulta, basta colocar seu código sql dentro da função q como no exemplo abaixo:

```python
q("""
    SELECT *
    FROM usuarios
""")
```

e o resultado será exibido na tela.

---

Nesta parte da atividade, você vai trabalhar com três tabelas:

- produtos
- usuarios
- vendas

Use-as para responder às perguntas a seguir.

---
#### Checagens
---

In [66]:
# Tabela de Usuários
q("""
    SELECT * FROM usuarios
    WHERE cod_usuario IS NOT NULL
    ORDER BY cod_usuario
""",n=10)

+-----------+-------------------+------------------+------------------+-------------------+
|cod_usuario|data_cadastro      |faixa_etaria      |cidade            |estado             |
+-----------+-------------------+------------------+------------------+-------------------+
|28         |2020-01-19 00:00:00|Entre 22 a 27 anos|Sertão            |Rio Grande do Sul  |
|69         |2020-07-24 00:00:00|Entre 16 a 21 anos|São Romão         |Minas Gerais       |
|89         |2020-03-18 00:00:00|Mais de 70 anos   |Campestre da Serra|Rio Grande do Sul  |
|124        |2020-12-17 00:00:00|Entre 28 a 36 anos|Curral de Dentro  |Minas Gerais       |
|153        |2020-02-29 00:00:00|Entre 37 a 49 anos|Santiago          |Rio Grande do Sul  |
|156        |2019-03-18 00:00:00|Entre 16 a 21 anos|Porto Nacional    |Tocantins          |
|176        |2021-03-19 00:00:00|Entre 28 a 36 anos|Guanambi          |Bahia              |
|185        |2021-03-11 00:00:00|Entre 16 a 21 anos|Martins           |Rio Grand

In [67]:
# Tabela de Vendas
q("""
    SELECT * FROM vendas
""",n=10)

+-----------+-----------+-------------------+----------+-------+
|cod_usuario|cod_produto|data_compra        |quantidade|valor  |
+-----------+-----------+-------------------+----------+-------+
|927        |10         |2020-10-19 00:00:00|4         |2799.6 |
|1544       |10         |2021-03-28 00:00:00|2         |1399.8 |
|2833       |2          |2020-09-13 00:00:00|20        |26915.6|
|2253       |14         |2021-06-21 00:00:00|1         |138.99 |
|1724       |10         |2021-07-10 00:00:00|6         |4199.4 |
|1534       |5          |2019-10-17 00:00:00|19        |8303.0 |
|1977       |14         |2020-05-08 00:00:00|2         |277.98 |
|1916       |3          |2020-12-01 00:00:00|14        |28693.0|
|2446       |1          |2021-06-02 00:00:00|17        |78045.3|
|2041       |4          |2021-07-25 00:00:00|19        |1119.1 |
+-----------+-----------+-------------------+----------+-------+
only showing top 10 rows



In [23]:
# Data Profiling
q("""
    WITH profiling AS (
        SELECT 
        'vendas' AS table_name,
        '-' AS column_name,
        'row_count' AS profiling,
        COUNT(*) AS value
        FROM vendas

        UNION ALL 

        SELECT 
        'usuarios' AS table_name,
        '-' AS column_name,
        'row_count' AS profiling,
        COUNT(*) AS value
        FROM usuarios

        UNION ALL

        SELECT 
        'vendas' AS table_name,
        'cod_usuario' AS column_name,
        'count_distinct' AS profiling,
        COUNT(DISTINCT cod_usuario) AS value
        FROM vendas
        WHERE
        'cod_usuario' IS NOT NULL

        UNION ALL 

        SELECT 
        'usuarios' AS table_name,
        'cod_usuario' AS column_name,
        'count_distinct' AS profiling,
        COUNT(DISTINCT cod_usuario) AS value
        FROM usuarios
        WHERE
        'cod_usuario' IS NOT NULL
        
        UNION ALL
        
        SELECT 
        'usuarios' AS table_name,
        'cod_usuario' AS column_name,
        'count_null' AS profiling,
        SUM(CASE WHEN ISNULL(cod_usuario) is true THEN 1 ELSE 0 END) AS value
        FROM usuarios
        GROUP BY 
        'vendas',
        'column_name',
        'count_null'

        UNION ALL
        
        SELECT 
        'vendas' AS table_name,
        'cod_usuario' AS column_name,
        'count_null' AS profiling,
        SUM(CASE WHEN ISNULL(cod_usuario) is true THEN 1 ELSE 0 END) AS value
        FROM vendas
        GROUP BY 
        'vendas',
        'column_name',
        'count_null'


        UNION ALL
        
        SELECT 
            'usuarios' AS table_name,
            'cod_usuario' AS column_name,
            'count_not_null' AS profiling,
            SUM(CASE WHEN ISNULL(cod_usuario) is true THEN 0 ELSE 1 END) AS value
        FROM 
            usuarios
        GROUP BY 
            'vendas',
            'column_name',
            'count_not_null'

        UNION ALL
        
        SELECT 
            'vendas' AS table_name,
            'cod_usuario' AS column_name,
            'count_not_null' AS profiling,
            SUM(CASE WHEN ISNULL(cod_usuario) is true THEN 0 ELSE 1 END) AS value
        FROM 
            vendas
        GROUP BY 
            'vendas',
            'column_name',
            'count_not_null'
    )
    SELECT * FROM profiling ORDER BY table_name, column_name, profiling;
    ;
""")

+----------+-----------+--------------+-----+
|table_name|column_name|profiling     |value|
+----------+-----------+--------------+-----+
|usuarios  |-          |row_count     |5570 |
|usuarios  |cod_usuario|count_distinct|3406 |
|usuarios  |cod_usuario|count_not_null|3408 |
|usuarios  |cod_usuario|count_null    |2162 |
|vendas    |-          |row_count     |20000|
|vendas    |cod_usuario|count_distinct|3397 |
|vendas    |cod_usuario|count_not_null|20000|
|vendas    |cod_usuario|count_null    |0    |
+----------+-----------+--------------+-----+



---
#### Data Cleaning
---

In [24]:
def limpa_tabela_sql(nome_tabela, query, export_dir):
    df = spark.sql(query)
    pd = df.toPandas()
    pd.to_csv(f"{export_dir}/{nome_tabela}.csv", index=False)
    print(f'Tabela: [{nome_tabela}] exportada com sucesso para [{export_dir}/{nome_tabela}.csv]')
    df.unpersist()

In [25]:
# Tabela Usuários
query = ("""
    WITH
    tab_usuarios_not_null AS (
        SELECT 
              usuarios.cod_usuario
            , usuarios.data_cadastro
            , usuarios.faixa_etaria
            , usuarios.cidade
            , usuarios.estado

        FROM 
            usuarios 
        WHERE 
            cod_usuario IS NOT NULL
    ),
    tab_usuarios_duplicated AS (
        SELECT
             cod_usuario
            ,COUNT(cod_usuario) AS count_cod_usuario
        FROM 
            tab_usuarios_not_null
        GROUP BY 
            cod_usuario
        HAVING
            COUNT(cod_usuario) > 1
    )

    SELECT 
          tab_usuarios_not_null.cod_usuario
        , tab_usuarios_not_null.data_cadastro
        , tab_usuarios_not_null.faixa_etaria
        , tab_usuarios_not_null.cidade
        , tab_usuarios_not_null.estado
    FROM 
        tab_usuarios_not_null
        LEFT JOIN tab_usuarios_duplicated ON tab_usuarios_not_null.cod_usuario = tab_usuarios_duplicated.cod_usuario
    WHERE
        tab_usuarios_duplicated.cod_usuario IS NULL
    ;
""")

# Chamando a função
limpa_tabela_sql('usuarios', query , 'bases_limpas')

Tabela: [usuarios] exportada com sucesso para [bases_limpas/usuarios.csv]


In [26]:
# Tabela Vendas
query = ("""                
    SELECT 
        *
    FROM 
        vendas
    ;
""")

# Chamando a função
limpa_tabela_sql('vendas', query , 'bases_limpas')

Tabela: [vendas] exportada com sucesso para [bases_limpas/vendas.csv]


In [27]:
# Tabela Produtos
query = ("""                
    SELECT 
        *
    FROM 
        produtos
    ;
""")

# Chamando a função
limpa_tabela_sql('produtos', query , 'bases_limpas')

Tabela: [produtos] exportada com sucesso para [bases_limpas/produtos.csv]


In [28]:
usuarios = cria_tabela("bases_limpas/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_limpas/produtos.csv", "produtos")
vendas = cria_tabela("bases_limpas/vendas.csv", "vendas")

In [31]:
# Conferindo a Tabela de Usuários após o Data Cleansing
q("""
SELECT 
        COUNT(*)
        FROM usuarios
        order by 1
""")

+--------+
|count(1)|
+--------+
|3404    |
+--------+



In [32]:
# Correspondencia entre as tabelas Usuários e Vendas
q("""
    WITH 
    tab_vendas_distinct_usuarios AS (
        SELECT DISTINCT cod_usuario FROM vendas
    ),
    tab_usuarios_distinct_usuarios AS (
        SELECT DISTINCT cod_usuario FROM usuarios
    )
    ,
    join_vendas_usuarios AS (
        SELECT 
              tab_vendas_distinct_usuarios.cod_usuario AS vendas_cod_usuario
            , tab_usuarios_distinct_usuarios.cod_usuario AS usuarios_cod_usuario
        FROM 
            tab_vendas_distinct_usuarios
            FULL OUTER JOIN tab_usuarios_distinct_usuarios ON tab_vendas_distinct_usuarios.cod_usuario = tab_usuarios_distinct_usuarios.cod_usuario
    )
    
    SELECT
        -- count(*)
         SUM(CASE WHEN ISNULL(vendas_cod_usuario) IS true THEN 0 ELSE 1 END) AS qt_cod_usuario_tab_vendas_not_null
        ,SUM(CASE WHEN ISNULL(usuarios_cod_usuario) IS true THEN 0 ELSE 1 END) AS qt_cod_usuario_tab_usuarios_not_null
        ,SUM(CASE WHEN vendas_cod_usuario = usuarios_cod_usuario IS true THEN 1 ELSE 0 END) AS qt_match_cod_usuario_vendas_usuarios
        -- ** ENTENDER ISSO: 
        -- ,SUM(IFNULL(vendas_cod_usuario,1)) AS qt_vendas_cod_usuario_not_null
        -- ,SUM(NVL(usuarios_cod_usuario,1)) AS qt_usuarios_cod_usuario_not_null
    FROM
        join_vendas_usuarios
""")

+----------------------------------+------------------------------------+------------------------------------+
|qt_cod_usuario_tab_vendas_not_null|qt_cod_usuario_tab_usuarios_not_null|qt_match_cod_usuario_vendas_usuarios|
+----------------------------------+------------------------------------+------------------------------------+
|3397                              |3404                                |66                                  |
+----------------------------------+------------------------------------+------------------------------------+



---
#### Perguntas sobre os dados
---

1) Qual foi a quantidade de vendas nos estados de Minas Gerais e São Paulo para cada ano e mês?

In [33]:
q("""
    WITH 
    qtd_vendas_estado AS (
        SELECT 
            usuarios.estado,
            DATE_FORMAT(vendas.data_compra,'y') as ano,
            DATE_FORMAT(vendas.data_compra,'MM') as mes,
            SUM(vendas.quantidade) as qt_vendas
        FROM 
            vendas
            LEFT JOIN usuarios ON usuarios.cod_usuario = vendas.cod_usuario
        WHERE 
            usuarios.estado IN ('Minas Gerais','São Paulo')
        GROUP BY 
            usuarios.estado,
            DATE_FORMAT(vendas.data_compra,'y'),
            DATE_FORMAT(vendas.data_compra,'MM')
        HAVING
            DATE_FORMAT(vendas.data_compra,'y') IS NOT NULL
            OR
            DATE_FORMAT(vendas.data_compra,'y') IS NOT NULL
    )
    
    SELECT 
        *
    FROM 
        qtd_vendas_estado
        PIVOT (
            IFNULL(SUM(qtd_vendas_estado.qt_vendas),'-')
            FOR mes IN ('01' AS jan, 
                        '02' AS fev,
                        '03' AS mar,
                        '04' AS abr,
                        '05' AS mai,
                        '06' AS jun,
                        '07' AS jul,
                        '08' AS ago,
                        '09' AS set,
                        '10' AS out,
                        '11' AS nov,
                        '12' AS dez)
        )
    ORDER BY
    1 , 2
    ;        

""")

+------------+----+---+---+---+---+---+---+---+---+---+---+---+---+
|estado      |ano |jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez|
+------------+----+---+---+---+---+---+---+---+---+---+---+---+---+
|Minas Gerais|2018|-  |-  |-  |-  |-  |10 |-  |16 |-  |20 |10 |39 |
|Minas Gerais|2019|20 |2  |-  |26 |11 |1  |14 |15 |27 |3  |1  |25 |
|Minas Gerais|2020|8  |10 |14 |1  |44 |18 |30 |-  |44 |8  |43 |18 |
|Minas Gerais|2021|21 |75 |6  |58 |22 |16 |61 |-  |-  |-  |-  |-  |
|São Paulo   |2018|-  |-  |-  |-  |-  |-  |9  |-  |-  |12 |-  |-  |
|São Paulo   |2019|-  |-  |-  |-  |-  |10 |-  |17 |-  |-  |-  |-  |
|São Paulo   |2020|6  |-  |1  |9  |-  |13 |-  |-  |22 |41 |-  |-  |
|São Paulo   |2021|-  |15 |1  |28 |22 |56 |96 |-  |-  |-  |-  |-  |
+------------+----+---+---+---+---+---+---+---+---+---+---+---+---+



2) Quais são os usuários por Estado que mais compraram em todo o período analisado e qual foi o número de compras realizadas, a quantidade total de itens comprados e valor total pago por usuário?

In [69]:
q("""
    WITH 
    
    -- Inicio: Maiores compradores por estado -- 
    q010_vendas_usuario_estado AS (
        SELECT 
            usuarios.estado,
            usuarios.cod_usuario,
            SUM(vendas.quantidade) AS qt_vendas
        FROM 
            vendas
            LEFT JOIN usuarios ON usuarios.cod_usuario = vendas.cod_usuario
        GROUP BY 
            usuarios.estado,
            usuarios.cod_usuario
        HAVING
            SUM(vendas.quantidade) IS NOT NULL
    ),
    
    q011_rank_vendas_usuario_estado AS (
        SELECT 
            q010.estado,
            q010.cod_usuario,
            q010.qt_vendas,
            RANK() OVER (PARTITION BY q010.estado
                             ORDER BY q010.estado, q010.qt_vendas
                        ) AS classificacao
        FROM 
            q010_vendas_usuario_estado AS q010
    ),

    q012_maiores_compradores_por_estado AS (
        SELECT 
            q011.estado,
            q011.cod_usuario,
            q011.qt_vendas,
            q011.classificacao
        FROM 
            q011_rank_vendas_usuario_estado AS q011
        WHERE
            q011.classificacao = 1
    ),    
    -- Fim : Maiores compradores por estado -- 
    
    
    -- Inicio: Quantidade de vendas por usuário -- 
    q020_vendas_por_usuario AS (
        SELECT 
            usuarios.estado,
            vendas.cod_usuario,
            COUNT(vendas.data_compra) AS count_compras_realizadas,
            SUM(vendas.quantidade) AS sum_quantidade_items_comprados,
            CAST(SUM(vendas.valor) AS DECIMAL(20,2)) as sum_valor_total_pago
        FROM 
            vendas
            LEFT JOIN usuarios ON vendas.cod_usuario = usuarios.cod_usuario
        GROUP BY
            usuarios.estado,
            vendas.cod_usuario
    )
    -- Fim: Quantidade de vendas por usuario -- 
    
    -- Tabela Final -- 
    SELECT 
        q012.estado,
        q012.cod_usuario,
        q020.count_compras_realizadas,
        q020.sum_quantidade_items_comprados,
        q020.sum_valor_total_pago,
        data_processamento
    FROM 
        q012_maiores_compradores_por_estado AS q012
        LEFT JOIN q020_vendas_por_usuario   AS q020 ON q012.cod_usuario = q020.cod_usuario AND q012.estado = q020.estado,
        (SELECT NOW() as data_processamento) 
    WHERE 
        q012.estado IS NOT NULL
    ORDER BY 
        1
""")

+-------------------+-----------+------------------------+------------------------------+--------------------+--------------------------+
|estado             |cod_usuario|count_compras_realizadas|sum_quantidade_items_comprados|sum_valor_total_pago|data_processamento        |
+-------------------+-----------+------------------------+------------------------------+--------------------+--------------------------+
|Amapá              |2186       |7                       |61                            |51359.34            |2022-07-31 11:43:08.317648|
|Bahia              |466        |4                       |33                            |13613.89            |2022-07-31 11:43:08.317648|
|Ceará              |1787       |8                       |79                            |121494.67           |2022-07-31 11:43:08.317648|
|Goiás              |1878       |4                       |54                            |96862.10            |2022-07-31 11:43:08.317648|
|Maranhão           |191        |7

3) Quais são os usuários que não fizeram nenhuma compra?

In [35]:
q("""
    WITH 
    -- q01 - Inicio - Quantidade de vendas por estado
    q01_qtd_vendas_estado AS (
        SELECT 
            usuarios.cod_usuario
        FROM 
            usuarios
            LEFT ANTI JOIN vendas ON usuarios.cod_usuario = vendas.cod_usuario
        GROUP BY 
            usuarios.cod_usuario
    ),
    -- q01 - Final - Quantidade de vendas por estado
    
    -- q99 - Inicio - Metadado
    q99_metadado AS (
        SELECT NOW() AS data_processamento
    )
    -- q99 - Final - Metadado
    
    -- Tabela Final --        
    SELECT 
          q01.cod_usuario
        , COUNT(q01.cod_usuario) OVER (PARTITION BY q99.data_processamento) AS quantidade_total_clientes_sem_compra
        , q99.data_processamento
    FROM 
        q01_qtd_vendas_estado AS q01,
        q99_metadado AS q99
    ;        
""")

+-----------+------------------------------------+--------------------------+
|cod_usuario|quantidade_total_clientes_sem_compra|data_processamento        |
+-----------+------------------------------------+--------------------------+
|19984      |3338                                |2022-07-31 05:30:05.853347|
|25517      |3338                                |2022-07-31 05:30:05.853347|
|41409      |3338                                |2022-07-31 05:30:05.853347|
|68090      |3338                                |2022-07-31 05:30:05.853347|
|87120      |3338                                |2022-07-31 05:30:05.853347|
|195413     |3338                                |2022-07-31 05:30:05.853347|
|200379     |3338                                |2022-07-31 05:30:05.853347|
|219523     |3338                                |2022-07-31 05:30:05.853347|
|246097     |3338                                |2022-07-31 05:30:05.853347|
|362827     |3338                                |2022-07-31 05:

4) Qual é o ticket médio (média de valor gasto) e o número total de usuários que fizeram pelo menos uma compra por faixa etária?

In [36]:
# A tabela de usuários só tem correspondencia na base de vendas para 66 usuários (~1.93% dos usuários únicos).
# Por isso é inviável fazer estimativas por 'faixa etária'.
# No entanto, podemos extrair a informação para os dados disponíveis, levando em consideração os usuários que possuem a faixa etária descrita.

q("""
    WITH 
    -- q01 - Inicio - Usuarios, Compras
    q01_usuarios_compras AS (
        SELECT  
             vendas.cod_usuario
            ,usuarios.faixa_etaria 
            ,vendas.quantidade
            ,vendas.valor
        FROM 
            vendas
            LEFT JOIN usuarios ON usuarios.cod_usuario = vendas.cod_usuario
    ),
    -- q01 - Final - Usuarios, Compras
    
    -- q99 - Inicio - Metadado
    q99_metadado AS (
        SELECT NOW() AS data_processamento
    )
    -- q99 - Final - Metadado
      
    -- Tabela Final --
    SELECT 
          q01.faixa_etaria
         ,COUNT(DISTINCT q01.cod_usuario) AS qtd_usuarios_compraram
         ,CAST(AVG(q01.valor) AS DECIMAL(10,2)) AS ticket_medio
         ,q99.data_processamento
    FROM 
         q01_usuarios_compras AS q01
        ,q99_metadado AS q99
    WHERE 
        faixa_etaria IS NOT NULL
    GROUP BY
         q01.faixa_etaria
        ,q99.data_processamento
    ORDER BY 
        q01.faixa_etaria
    ;        
""")

+------------------+----------------------+------------+--------------------------+
|faixa_etaria      |qtd_usuarios_compraram|ticket_medio|data_processamento        |
+------------------+----------------------+------------+--------------------------+
|Entre 10 a 15 anos|7                     |8890.13     |2022-07-31 05:30:13.691028|
|Entre 16 a 21 anos|11                    |12725.95    |2022-07-31 05:30:13.691028|
|Entre 22 a 27 anos|10                    |12203.02    |2022-07-31 05:30:13.691028|
|Entre 28 a 36 anos|9                     |8233.04     |2022-07-31 05:30:13.691028|
|Entre 37 a 49 anos|6                     |16727.19    |2022-07-31 05:30:13.691028|
|Entre 50 a 61 anos|12                    |10849.48    |2022-07-31 05:30:13.691028|
|Entre 62 a 70 anos|7                     |10012.63    |2022-07-31 05:30:13.691028|
|Mais de 70 anos   |4                     |11303.10    |2022-07-31 05:30:13.691028|
+------------------+----------------------+------------+--------------------

In [20]:
spark.stop()

# PySpark

**setup**:

In [1]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

mkdir: cannot create directory ‘bases_teste’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1039  100  1039    0     0   8725      0 --:--:-- --:--:-- --:--:--  8805
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533k  100  533k    0     0  2507k      0 --:--:-- --:--:-- --:--:-- 2518k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211k  100  211k    0     0   696k      0 --:--:-- --:--:-- --:--:--  694k


In [13]:
# Setup Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

22/07/31 04:35:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/07/31 04:35:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [14]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

Responda às perguntas a seguir utilizando **Spark DATAFRAMES**.

1) Qual foi o total de compras realizadas, o total de itens comprados e a receita total obtida em todo o período analisado?

In [15]:
# from pyspark.sql.functions import avg, format_number, round

# Schema 'vendas'
vendas.printSchema()

# Somas
vendas.groupBy() \
    .sum("quantidade", "valor") \
    .show(truncate=False)
    

root
 |-- cod_usuario: integer (nullable = true)
 |-- cod_produto: integer (nullable = true)
 |-- data_compra: timestamp (nullable = true)
 |-- quantidade: integer (nullable = true)
 |-- valor: double (nullable = true)

+---------------+--------------------+
|sum(quantidade)|sum(valor)          |
+---------------+--------------------+
|209149         |2.1584900165999958E8|
+---------------+--------------------+



2) Quais são os 3 produtos mais comprados dos estados da região Sul e Sudeste, a quantidade de itens comprados, o valor total pago e a média de preço paga?

In [65]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col


# Join das Tabelas Vendas, Usuarios e Produtos
df_vendas_usuarios_produtos = vendas.join(usuarios, (vendas.cod_usuario == usuarios.cod_usuario)) \
        .join(produtos, ['cod_produto'])

# Agregação
df_vendas_usuarios_produtos = df_vendas_usuarios_produtos.groupBy("estado","cod_produto") \
    .sum("quantidade","valor") \
    .withColumnRenamed("sum(quantidade)", "sum_quantidade") \
    .withColumnRenamed("sum(valor)", "sum_valor")

# Classificação de Vendas por Estado e Produto
windowSpec  = Window.partitionBy("estado").orderBy("sum_quantidade")

regiao_sudeste = ['Espirito Santo', 'Minas Gerais', 'São Paulo', 'Rio de Janeiro']

df_vendas_usuarios_produtos.withColumn("row_number",row_number().over(windowSpec)) \
    .filter(col("estado").isin(regiao_sudeste)) \
    .filter(col("row_number") <= 3) \
    .withColumn('media_preco_pago', df_vendas_usuarios_produtos['sum_valor'] / (df_vendas_usuarios_produtos['sum_quantidade'])) \
    .show(n=10000, truncate=False)



+------------+-----------+--------------+------------------+----------+-----------------+
|estado      |cod_produto|sum_quantidade|sum_valor         |row_number|media_preco_pago |
+------------+-----------+--------------+------------------+----------+-----------------+
|Minas Gerais|16         |6             |30.0              |1         |5.0              |
|Minas Gerais|11         |7             |349.29999999999995|2         |49.89999999999999|
|Minas Gerais|13         |9             |89.1              |3         |9.899999999999999|
|São Paulo   |9          |1             |389.1             |1         |389.1            |
|São Paulo   |17         |2             |1168.2            |2         |584.1            |
|São Paulo   |11         |6             |299.4             |3         |49.9             |
+------------+-----------+--------------+------------------+----------+-----------------+



3) Para cada produto, quantos usuários fizeram pelo menos uma compra desse produto e qual é o valor mínimo e máximo pago por eles?

4) Aplique um desconto de 10% em todas as vendas dos usuários que fizeram pelo menos 3 compras de produtos na mesma categoria, a partir da 4ª compra realizada. Exiba apenas os usuários que terão o desconto aplicado, mantendo todas as compras, o valor original e o valor com o desconto aplicado.

In [111]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, when


# Join das Tabelas Vendas e Produtos
df_vendas_produtos = vendas.join(produtos, ['cod_produto'])

# Classificação das compras por usuario e categoria
windowSpec  = Window.partitionBy("cod_usuario","categoria_produto").orderBy("data_compra")
df_vendas_produtos = df_vendas_produtos.withColumn("numero_da_compra",row_number().over(windowSpec))

# Calcula 10% de desconto a partir da 4ª compra
df_vendas_produtos = df_vendas_produtos.withColumn("valor_com_desconto", when(df_vendas_produtos.numero_da_compra > 3, df_vendas_produtos.valor - df_vendas_produtos.valor * 0.1) \
        .otherwise(df_vendas_produtos.valor))

# Filtra os resultados 
df_vendas_produtos = df_vendas_produtos.filter(col("numero_da_compra") > 3 )

# Exibe o resultado
df_vendas_produtos.show()


+-----------+-----------+-------------------+----------+--------+--------------------+--------------------+-------------+----------------+------------------+
|cod_produto|cod_usuario|        data_compra|quantidade|   valor|        nome_produto|   categoria_produto|valor_produto|numero_da_compra|valor_com_desconto|
+-----------+-----------+-------------------+----------+--------+--------------------+--------------------+-------------+----------------+------------------+
|          1|          3|2021-07-20 00:00:00|         8| 36727.2|Notebook Asus Int...|          Tecnologia|       4590.9|               4|33054.479999999996|
|          8|          6|2021-03-23 00:00:00|         2|  439.14| Cafeteira Nespresso|    Eletrodomesticos|       219.57|               4|           395.226|
|          1|         16|2021-02-16 00:00:00|         9| 41318.1|Notebook Asus Int...|          Tecnologia|       4590.9|               4|          37186.29|
|         17|         16|2021-05-23 00:00:00|       

In [12]:
spark.stop()

# FIM!